<a href="https://colab.research.google.com/github/SultokTheF/QazQA/blob/main/Test_Kaz_QA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install transformers
!pip install sentencepiece
!pip install nltk

In [ ]:
import pandas as pd, numpy as np, re, string
import unicodedata

import torch
from torch.utils.data import DataLoader

from transformers import T5Tokenizer, T5TokenizerFast
from transformers import T5ForConditionalGeneration

from tqdm import tqdm
import sentencepiece

from sklearn.model_selection import train_test_split
from datetime import datetime

from nltk.translate.bleu_score import sentence_bleu, corpus_bleu
from nltk.translate.meteor_score import meteor_score
from collections import Counter

max_text_length   = 256

In [ ]:
tokenizer = T5Tokenizer.from_pretrained('Kyrmasch/t5-kazakh-qa')
model     = T5ForConditionalGeneration.from_pretrained('Kyrmasch/t5-kazakh-qa')

In [ ]:
context = ""
question = "Ертiс ауыл ма, өзен ме?"

encoded = tokenizer.encode_plus(context, question, max_length=128, pad_to_max_length=True, truncation=True, return_tensors="pt")
input_ids = encoded["input_ids"].to('cpu')
attention_mask = encoded["attention_mask"].to('cpu')
output = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=128)
answer = ''.join([tokenizer.decode(ids, skip_special_tokens=True) for ids in output])

print(answer)

өзен


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
gold = pd.read_csv("/content/drive/MyDrive/Data/KAZ-test-T5/444 gold.csv", sep=';')

In [ ]:
gold

,context,question,answer,gen_answer,score
0,Коала - бұл Австралияда ғана өмір сүретін және...,Коала қайда өмір сүреді?,Коала тек Австралияда өмір сүреді,Австралия,0.0
1,"Жапония - ерекше мәдениетімен, технологиясымен...",Жапония несімен танымал?,"Жапония тауларымен, жанартауларымен, ерекше мә...","ерекше мәдениетімен, технологиясымен және таул...",0.0
2,Раушан гүлдері әртүрлі түстерде гүлдейтін әдем...,Эфир майларын жасауға қолданылатын гүл қалай а...,Раушан гүлдері,Раушан гүлдері,0.0
3,Кактустар құрғақ климатқа бейімделген және сус...,Кактус қандай климатқа бейімделген?,Кактустар құрғақ климатқа бейімделген,құрғақ,0.0
4,Секвоялар - әсерлі өлшемдерге жететін және 200...,Секвоялар қай жерде өседі?,Солтүстік Американың батыс бөлігінде өседі,Солтүстік Американың батыс бөлігі,0.0
...,...,...,...,...,...
439,Ақынның шығармашылық-күрескерлік қызметін музы...,"1941 жылы жазылған ""Абай"" атты 5 бөлімді лирик...",Ахмет Жұбанов,Ахмет Жұбанов,0.0
440,"Абайдың ""Мың бір түн"" ертегісінің желісімен жа...",Ұлы ақынның аяқталмай қалған дастаны қалай ата...,Әзім әңгімесі,Әзім әңгімесі,0.0
441,"Ұлы ойшылдың 150 жылдығына орай Семейде ""Абай ...",Абайдың 150 жылдығы қарсаңында Семейде құрылға...,Абай қоры,Абай қоры,0.0
442,Абайдың 1889 жылы жазған бұл өлеңі әрқайсысы 4...,"Абайдың ""Айттым сәлем, қалам қас..."" өлеңі қан...",10,10,0.0


In [ ]:
import random
# Define the size of the random subset you want to extract
subset_size = 10

# Get a list of random indices
random_indices = random.sample(range(len(gold)), subset_size)

# Extract the subset of the DataFrame using the random indices
sample = gold.iloc[random_indices]

In [ ]:
sample

,context,question,answer
16,Жасанды интеллектті дамытудың түпкі мақсаты-де...,Жасанды интеллект не талдауға қолданылады?,Деректердің үлкен көлемін талдауға
348,Қазақ АКСР Пошта және Телеграф халық комиссари...,"Қазақ радиосы қай жылы, қай қаладан хабар тара...",1921 жылы Орынбордан
295,Үлкен тосқауыл Рифі жер шарындағы ең ірі маржа...,Жер шарындағы ең ірі маржан түзінділері,Үлкен тосқауыл Рифі
204,Жер шарындағы ең биік тау шыңы Гималай тауында...,Жер шарындағы ең биік тау шыңы калай аталады?,Гималай тауындағы Джомолунгма шыңы
249,Қазақстандағы су электр станциясы Қапшағай жән...,Қазақстандағы су электр станциясы қай аймақта ...,"Қапшағай, Шардара"
226,Шығыс Қазақстанның экономикасында жетекші рөл ...,Шығыс Қазақстанның экономикасында жетекші рөл ...,түсті металлургия
106,Пластидтер – тек өсімдік жасушасына ғана тән д...,Пластидтер нешеге бөлінеді?,3-ке.
135,"Сүйектің құрамында 50% су, 12,5% нәруыз тектес...","Сүйекке серпінді, иілгіштік қасиет беретін за...",Оссеин
77,"Театрда актер, режиссер, суретші, композитор, ...","Композитор, гример актер қайда ұжым болып еңбе...",Театрда
341,Мұқағали Мақатаев бірнеше кісімен әңгімелесіп ...,Мұқағали Мақатаев жары Лашынға 18 жасында үйле...,Ат үстіндегі Лашынды көріп қалады


In [ ]:
answer

'жалған дәрілердің жеткізу тізбегіне неғұрлым мөлдір ету'

In [ ]:
cuda = torch.cuda.is_available()
device = torch.device('cuda') if cuda else torch.device('cpu')
model.to(device)
model.eval()
def test(context, question):
    encoded = tokenizer.encode_plus(context,
                                    question,
                                    max_length=max_text_length,
                                    pad_to_max_length=True,
                                    truncation=True,
                                    return_tensors="pt")
    input_ids = encoded["input_ids"].to(device)
    attention_mask = encoded["attention_mask"].to(device)
    with torch.no_grad():
        output = model.generate(input_ids=input_ids,
                                attention_mask=attention_mask,
                                max_length=max_text_length)
    return [tokenizer.decode(ids, skip_special_tokens=True) for ids in output]

In [ ]:
%%capture
correct = 0
total   = 0

gold['gen_answer'] = ''
gold['score']      = 0.0

for i, row in gold.iterrows():
    context  = row['context']
    question = row['question']
    answer   = row['answer']

    # Combine the context and question into a single string
    gen_answer = ''.join(test(context, question))
    gold.at[i, 'gen_answer'] = gen_answer


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


In [ ]:
def normit(s):
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation+'«'+'»'+'"'+'”'+'“')
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    def clean_text(text):
        return(text.replace("<pad>", "").replace("</s>", ""))

    return white_space_fix(clean_text(remove_articles(remove_punc(lower(s)))))

def f1_score(prediction, ground_truth):
    prediction_tokens   = normit(prediction).split()
    ground_truth_tokens = normit(ground_truth).split()
    common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
    num_same = sum(common.values())
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(prediction_tokens)
    recall = 1.0 * num_same / len(ground_truth_tokens)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1

def exact_match_score(prediction, ground_truth):
    return (normit(prediction) == normit(ground_truth))

def evaluate(predictions, ground_truths):
    total_f1 = em = total = 0
    for prediction, ground_truth in zip(predictions, ground_truths):
        total += 1
        em += exact_match_score(prediction, ground_truth)
        total_f1 += f1_score(prediction, ground_truth)

    em_score = 100.0 * em / total
    f1_score_avg = 100.0 * total_f1 / total
    return {'EM': em_score, 'F1': f1_score_avg}


In [ ]:
predictions   = gold['gen_answer']
ground_truths = gold['answer']

result = evaluate(predictions, ground_truths)
print(result)

{'EM': 40.090090090090094, 'F1': 67.81052032933172}


In [ ]:
for i in range(444):
  print(normit(predictions[i]), ' ', normit(ground_truths[i]), '  ', exact_match_score(predictions[i], ground_truths[i]))

австралия   коала тек австралияда өмір сүреді    False
ерекше мәдениетімен технологиясымен және таулары жанартаулары мен әдемі бақтары бар тамаша табиғатымен   жапония тауларымен жанартауларымен ерекше мәдениетімен танымал    False
раушан гүлдері   раушан гүлдері    True
құрғақ   кактустар құрғақ климатқа бейімделген    False
солтүстік американың батыс бөлігі   солтүстік американың батыс бөлігінде өседі    False
байкал   байкал    True
перу және боливия шекарасында   перу мен боливия шекарасында орналасқан    False
су эрозиясы   каньон су эррозиясынан пайда болады    False
пингвиндер   пингвин    False
6 метр   жирафтардың биіктігі 6 метрге дейін жете алады    False
африка   ніл африкада орналасқан    False
перғауындар   перғауындардың құрметіне    False
8000 км   ұзындығы 8000 км асады    False
қыста демалуға және шаңғы тебуге арналған танымал орын   қыста демалуға және шаңғы тебуімен танымал    False
амазонка тропикалық орманы   амазонка тропикалық орманы    True
әртүрлі рәсімдер мен

In [ ]:
print(predictions)

0                                              Австралия
1      ерекше мәдениетімен, технологиясымен және таул...
2                                         Раушан гүлдері
3                                                 құрғақ
4                      Солтүстік Американың батыс бөлігі
                             ...                        
439                                        Ахмет Жұбанов
440                                        Әзім әңгімесі
441                                            Абай қоры
442                                                   10
443                                       Ришат Абдуллин
Name: gen_answer, Length: 444, dtype: object


In [ ]:
pd.set_option('max_colwidth', 200)

gold[["question", "answer", "gen_answer", "score"]][300:349]

In [ ]:
context = """Қар деп атмосфералық жауын-шашынды айтады. Қар ұсақ мұз кристалдарынан тұрады.
Қазақстан аймағында қар қазан-қараша айларынан бастап түседі.
Қазақстандағы қар жамылғысының қалыңдығы 50-60 см-ге жетеді. Қардың түсі ақ."""

question_1 = "Қар дегеніміз не?"
question_2 = "Қар неден тұрады?"
question_3 = "Қардың түсі қандай?"
question_4 = "Қазақстанда қар жамылғысының қалыңдығы қанша?"

answer_1 = ''.join(test(context, question_1))
answer_2 = ''.join(test(context, question_2))
answer_3 = ''.join(test(context, question_3))
answer_4 = ''.join(test(context, question_4))


/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2339: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
print(context, "\n")

print(question_1, "---> ", answer_1)
print(question_2, "---> ", answer_2)
print(question_3, "---> ", answer_3)
print(question_4, "---> ", answer_4)

Қар деп атмосфералық жауын-шашынды айтады. Қар ұсақ мұз кристалдарынан тұрады. 
Қазақстан аймағында қар қазан-қараша айларынан бастап түседі. 
Қазақстандағы қар жамылғысының қалыңдығы 50-60 см-ге жетеді. Қардың түсі ақ. 

Қар дегеніміз не? --->  атмосфералық жауын-шашынды
Қар неден тұрады? --->  ұсақ мұз кристалдары
Қардың түсі қандай? --->  ақ
Қазақстанда қар жамылғысының қалыңдығы қанша? --->  5060


In [ ]:
gold.to_csv("Gold_with_answers.csv", sep=';', encoding='utf-16')